In [2]:
import qiskit

import numpy as np
from qiskit.visualization import plot_state_qsphere, plot_bloch_multivector

def Rz(theta):
    return np.array([[np.e**(-1j*theta/2), 0],[0, np.e**(1j*theta/2)]])

def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],[np.sin(theta/2), np.cos(theta/2)]])

1. Dado uma matriz unitária V, determine os números reais `alpha`, `beta`, `gamma` e `delta` de modo que
```python
V = e^{1j*alpha} * Rz(beta) @ Ry(gamma) @ Rz(delta).
```

In [48]:
U = np.array([[(1+1j)/2, (1-1j)/2], [(1-1j)/2, (1+1j)/2]])
#U = [[(1)/np.sqrt(2), (1)/np.sqrt(2)],[(1)/np.sqrt(2), -1*(1)/np.sqrt(2)]]
# -----------------
# Seu código aqui

x00 = np.angle(U[0][0])
x01 = np.angle(U[0][1])
x10 = np.angle(U[1][0])
x11 = np.angle(U[1][1])

A = np.array([[1, -0.5, -0.5], [1, -0.5, 0.5], [1, 0.5, -0.5]])
B = np.array([x00, x01 + np.pi, x10])
X = np.linalg.inv(A).dot(B)    

alpha = X[0]
beta = X[1]
gamma = np.arccos(np.abs(U[0][0]))*2
delta = X[2]

In [49]:
print(np.e**(alpha*1j) * Rz(beta) @ Ry(gamma) @ Rz(delta))

[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]


2. Dada uma matriz unitária V, construa um circuito para o operador V controlado. Nesta questão você pode utilizar apenas operadores sobre um qubit e cx.

In [13]:
from qiskit import Aer, execute, ClassicalRegister, QuantumRegister, QuantumCircuit

V1 = np.array([[(1+1j)/2, (1-1j)/2], [(1-1j)/2, (1+1j)/2]])
V2 = [[(1)/np.sqrt(2), (1)/np.sqrt(2)],[(1)/np.sqrt(2), -1*(1)/np.sqrt(2)]]
# -----------------
# Seu código aqui
def q2(V):
    #Decomponsição de V

    x00 = np.angle(V[0][0])
    x01 = np.angle(V[0][1])
    x10 = np.angle(V[1][0])
    x11 = np.angle(V[1][1])

    A = np.array([[1, -0.5, -0.5], [1, -0.5, 0.5], [1, 0.5, -0.5]])
    B = np.array([x00, x01 + np.pi, x10])
    X = np.linalg.inv(A).dot(B)    

    alpha = X[0]
    beta = X[1]
    gamma = np.arccos(np.abs(V[0][0]))*2
    delta = X[2]

    #-------Matrizes------#
    A = Rz(beta)*Ry(gamma/2)
    B = Ry(-1*gamma/2)*Rz(-1*(delta + beta)/2)
    C = Rz((delta-beta)/2)

    #-------Circuito------#
    circuito = qiskit.QuantumCircuit(2)
    #C
    circuito.rz((delta-beta)/2, 1)
    #cx
    circuito.cx(0,1)
    #B
    circuito.rz(-1*(delta + beta)/2, 1)
    circuito.ry(-1*gamma/2, 1)
    #cx
    circuito.cx(0,1)
    #A
    circuito.ry(gamma/2, 1)
    circuito.rz(beta, 1)
    circuito.draw()
    #Operador [[1,0],[0,e**(1j*alpha)]] -> U3=0,0,alpha
    circuito.u3(theta=0, phi=0, lam=alpha, qubit=0)
    return circuito

In [14]:
circuito = q2(V2)
circuito.draw()

┌──────────────┐         
q_0: ──────────────■──────────────────────────────■──┤ U3(0,0,pi/2) ├─────────
     ┌──────────┐┌─┴─┐┌───────────┐┌───────────┐┌─┴─┐└─┬──────────┬─┘┌───────┐
q_1: ┤ RZ(pi/2) ├┤ X ├┤ RZ(-pi/2) ├┤ RY(-pi/4) ├┤ X ├──┤ RY(pi/4) ├──┤ RZ(0) ├
     └──────────┘└───┘└───────────┘└───────────┘└───┘  └──────────┘  └───────┘

#https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html

3. Utilizando apenas operadores sobre um qubit e cx, construa um circuito quântico que tenha a ação do operador Toffoli.


In [27]:
def toffoli():
    '''
    Lab2 - questão 3
    :param n: número de controles
    :param V:
    :return: circuito quântico com n+1 qubits + n-1 qubits auxiliares
            que aplica o operador nCV.
    '''
    controles = qiskit.QuantumRegister(2)
    alvo = qiskit.QuantumRegister(1)

    circuito = qiskit.QuantumCircuit(controles, alvo)

    #------------------------
    # Seu código aqui
    # ------------------------
    '''
    circuito.cu1(np.pi, controles[1], alvo)
    circuito.cx(controles[0],controles[1])
    circuito.cu1(-np.pi, controles[1], alvo)
    circuito.cx(controles[0],controles[1])
    circuito.cu1(np.pi,controles[0],alvo)
    '''
    circuito.ccx(controles[0], controles[1], alvo)
    return circuito

In [41]:
circuito = toffoli()
circuito.draw()

q19_0: ───────■──────────■───■───
            ┌─┴─┐      ┌─┴─┐ │   
q19_1: ─■───┤ X ├─■────┤ X ├─┼───
        │pi └───┘ │-pi └───┘ │pi 
q20_0: ─■─────────■──────────■───

4. Dado um vetor real com norma 1 e dimensão 8, construa um circuito que inicialize um estado quântico com amplitudes correspondendo às coordenadas do vetor.


In [138]:
v8D = np.array([np.sqrt(0.03),np.sqrt(0.07),np.sqrt(0.15),np.sqrt(0.05),np.sqrt(0.1),np.sqrt(0.3),np.sqrt(0.2),np.sqrt(0.1)])

def inicializa_3qubits(vetor_dimensao8):
    '''
    Lab2 - questão 4
    '''
    arrays = np.split(vetor_dimensao8, 4)

    #Arvore de Estados
    aux_l = np.empty((4))

    i=0
    for array in arrays:
        aux_l[i] = np.linalg.norm(array)
        i+= 1

    i=0
    s_aux_l = np.empty((2))
    s_arrays = np.split(aux_l, 2)
    for s_array in s_arrays: 
        s_aux_l[i] = np.linalg.norm(s_array)
        i+=1

    t_aux_l = np.linalg.norm(s_aux_l)

    #Definindo os alphas
    alpha0 = 2*np.arcsin(np.sqrt(s_aux_l[1])/np.sqrt(t_aux_l))

    alpha1 = 2*np.arcsin(np.sqrt(aux_l[1])/np.sqrt(s_aux_l[0]))
    alpha2 = 2*np.arcsin(np.sqrt(aux_l[3])/np.sqrt(s_aux_l[1]))

    alpha3 = 2*np.arcsin(np.sqrt(vetor_dimensao8[1])/np.sqrt(aux_l[0]))
    alpha4 = 2*np.arcsin(np.sqrt(vetor_dimensao8[3])/np.sqrt(aux_l[1]))
    alpha5 = 2*np.arcsin(np.sqrt(vetor_dimensao8[5])/np.sqrt(aux_l[2]))
    alpha6 = 2*np.arcsin(np.sqrt(vetor_dimensao8[7])/np.sqrt(aux_l[3]))

    #Circuito
    
    qr = qiskit.QuantumRegister(3, 'q')
    circuito = qiskit.QuantumCircuit(qr)
    
    # ------------------------
    # Seu código aqui
    # ------------------------

    circuito.ry(alpha0,qr[0])
    
    circuito.x(qr[0])
    circuito.cry(alpha1,qr[0],qr[1]) #if bit[0] = 0 faz o controle no bit[1]
    
    circuito.x(qr[0])
    circuito.cry(alpha2,qr[0],qr[1]) #if bit[0] = 1 faz o controle no bit[1]
    
    circuito.x(qr[1])
    circuito.x(qr[0])
    circuito.mcry(theta=alpha3, q_controls=qr[0:2], q_target=qr[2], mode='noancilla',q_ancillae=None) #multi control bit; if bit[0] e bit[1] ambos são 0, entao faz o ry no bit 2
    
    circuito.x(qr[1])
    circuito.mcry(theta=alpha4, q_controls=qr[0:2], q_target=qr[2], mode='noancilla',q_ancillae=None) #multi control bit; if bit[0]=0 e bit[1]=1 entao faz o ry no bit 2

    circuito.x(qr[0])
    circuito.x(qr[1])
    circuito.mcry(theta=alpha5, q_controls=qr[0:2], q_target=qr[2], mode='noancilla',q_ancillae=None) #multi control bit; if bit[0]=1 e bit[1]=0 entao faz o ry no bit 2

    circuito.x(qr[1])
    circuito.mcry(theta=alpha6, q_controls=qr[0:2], q_target=qr[2], mode='noancilla',q_ancillae=None) #multi control bit; if bit[0]=1 e bit[1]=1 entao faz o ry no bit 2

    return circuito

In [139]:
circuito = inicializa_3qubits(v8D)
circuito.draw()

┌────────────┐┌───┐              ┌───┐              ┌───┐»
q_0: ┤ RY(2.3095) ├┤ X ├──────■───────┤ X ├──────■───────┤ X ├»
     └────────────┘└───┘┌─────┴──────┐└───┘┌─────┴──────┐├───┤»
q_1: ───────────────────┤ RY(2.2562) ├─────┤ RY(1.8853) ├┤ X ├»
                        └────────────┘     └────────────┘└───┘»
q_2: ─────────────────────────────────────────────────────────»
                                                              »
«                       ┌───┐                   ┌───┐                       »
«q_0: ──────────────────┤ X ├─────────■─────────┤ X ├────────■──────────────»
«                       └─┬─┘         │         └─┬─┘        │         ┌───┐»
«q_1: ────────■───────────■───────────┼───────────■──────────┼─────────┤ X ├»
«     ┌───────┴────────┐     ┌────────┴────────┐     ┌───────┴────────┐└───┘»
«q_2: ┤ U3(1.1547,0,0) ├─────┤ U3(-1.1547,0,0) ├─────┤ U3(1.1547,0,0) ├─────»
«     └────────────────┘     └─────────────────┘     └────────────────┘     »
«                     ┌───┐                 ┌───┐                ┌───┐»
«q_0: ────────────────┤ X ├────────■────────┤ X ├───────■────────┤ X ├»
«                     └─┬─┘        │        └─┬─┘       │        ├───┤»
«q_1: ───────■──────────■──────────┼──────────■─────────┼────────┤ X ├»
«     ┌──────┴───────┐     ┌───────┴───────┐     ┌──────┴───────┐└───┘»
«q_2: ┤ U3(pi/4,0,0) ├─────┤ U3(-pi/4,0,0) ├─────┤ U3(pi/4,0,0) ├─────»
«     └──────────────┘     └───────────────┘     └──────────────┘     »
«                       ┌───┐                   ┌───┐                       »
«q_0: ──────────────────┤ X ├─────────■─────────┤ X ├────────■──────────────»
«                       └─┬─┘         │         └─┬─┘        │         ┌───┐»
«q_1: ────────■───────────■───────────┼───────────■──────────┼─────────┤ X ├»
«     ┌───────┴────────┐     ┌────────┴────────┐     ┌───────┴────────┐└───┘»
«q_2: ┤ U3(1.1961,0,0) ├─────┤ U3(-1.1961,0,0) ├─────┤ U3(1.1961,0,0) ├─────»
«     └────────────────┘     └─────────────────┘     └────────────────┘     »
«                        ┌───┐                    ┌───┐                   
«q_0: ───────────────────┤ X ├─────────■──────────┤ X ├─────────■─────────
«                        └─┬─┘         │          └─┬─┘         │         
«q_1: ─────────■───────────■───────────┼────────────■───────────┼─────────
«     ┌────────┴────────┐     ┌────────┴─────────┐     ┌────────┴────────┐
«q_2: ┤ U3(0.86306,0,0) ├─────┤ U3(-0.86306,0,0) ├─────┤ U3(0.86306,0,0) ├
«     └─────────────────┘     └──────────────────┘     └─────────────────┘